# Parallel Python

In this section we briefly introduce three approaches for parallel computing in Python: `ipyparallel`, `multiprocessing`, and `mpi4py`.

> Many excellent resources on parallel Python exist on the web, and some have been used as inspiration for the material presented here. In particular, the following resources are recommended:
> - https://github.com/dvalters/RSE18-Python-Parallel-workshop
> - https://swcarpentry.github.io/python-intermediate-mosquitoes/04-multiprocessing.html

## Why would you want to do parallel programming in Python? 

Traditionally, Python is considered to not support parallel programming very well, and "proper" parallel programming should be left to "heavy-duty" languages such as Fortran or C/C++ where libraries or standards such as OpenMP and MPI can be utilised. 

For large scale, massively-parallel applications, this is probably still the case, but a rich variety of libraries and packages have been developed outside the core Python language, so parallel programming is now much better supported.

### Before spending a lot of time parallelizing your Python code...
- If your Python code is running too slow, there are more straightforward ways to speed it up:
    - Begin by identifying the performance bottlenecks in the code - **profile before optimizing!**
    - Use fast numerical packages like [Numpy](http://www.numpy.org/).
    - Use a just-in-time (JIT) compiler like [Numba](https://numba.pydata.org/).
    - Use C-extensions from [Cython](http://cython.org/).
    - Rewrite the performance-critical functions in C, and import them into Python.
    - Any of these methods could speed up Python code by orders of magnitude!
- So why bother with parallelizing Python?
    - Perhaps you're already using `Numpy`, `Numba` and `Cython` for the most compute-intensive parts of your code.
    - Perhaps you have a problem that is particularly suitable for parallelization, e.g. a large dataset that can be processed independently in chunks.

## Global interpreter lock

- The most common implementation of Python (interpreter/executable that runs your Python code) is called CPython.
- CPython doesn't support using threads well, because it's been written to assume that individual Python programs are serial.
- CPython implements something called the Global Interpreter Lock (GIL) that protects access to Python objects, preventing multiple threads executing Python bytecode through the Python interpreter at once.
- Subsequent developments in Python have come to rely on the GIL being present, so removing it in future versions of Python is unlikely.
- Parallel approaches to Python are normally based around running multiple instances of the Python interpreter, each with its own copy of the the code being run and each with its own separate GIL.

## IPython for parallel computing

> Adapted from the [official documentation](https://ipyparallel.readthedocs.io/en/)

IPython abstracts out parallelism in a general way, supporting many different styles of parallelism:

- Single program, multiple data (SPMD) parallelism
- Multiple program, multiple data (MPMD) parallelism
- Message passing using MPI
- Task farming
- Data parallel
- Combinations of these approaches
- Custom user-defined approaches

Most importantly, IPython and the `ipyparallel` package enables all types of parallel applications to be developed, executed, debugged, and monitored *interactively*.

The following are some example use cases:

- Quickly parallelize algorithms that are embarrassingly parallel using a number of simple approaches. Many simple things can be parallelized interactively in one or two lines of code.
- Steer traditional MPI applications on a supercomputer from an IPython session on your laptop.
- Analyze and visualize large datasets (that could be remote and/or distributed) interactively using IPython and tools like matplotlib.
- Develop, test and debug new parallel algorithms (that may use MPI) interactively.
- Tie together multiple MPI jobs running on different systems into one giant distributed and parallel system.
- Run a set of tasks on a set of CPUs using dynamic load balancing.


### Configuration

You may need to run the following commands to enable IPython Clusters in Jupyter:
```bash
$ jupyter serverextension enable --user --py ipyparallel 
$ jupyter nbextension install --user --py ipyparallel 
$ jupyter nbextension enable --py ipyparallel
```

### Getting started

IPython cluster for parallel computing can be started from the Jupyter notebook start page. Go to "IPython clusters", choose number of engiens (e.g. 4), and click "Start". 

In Jupyter notebook, type

In [ ]:
import ipyparallel as ipp
client = ipp.Client()
print("Number of ipyparallel engines:", len(client.ids))

The `ipyparallel` engines can be controlled by the `DirectView` instance, which has a ``map_sync`` function for distributing workloads across the engines.

In [ ]:
dview = client[:]
print(dview)

Suppose we want to calculate the square of 10 integers. We can first define a function and then calculate the squares serially

In [ ]:
def square(x):
    return x*x

output = [square(x) for x in range(1,11)]
print(output)

With `ipyparallel` it is handy to do this via `map_sync`

In [ ]:
output = dview.map_sync(square, range(1,11))
print(output)

The syntax for `map_sync` is straightforward - it accepts the function and a list of input arguments.

### Example: distance between cities

To further demonstrate parallel computing in Python, we introduce a slightly more complicated example.

In this example we provide the latitude and longitude of a list of cities. The task is to

+ calculate the distances between all pairs of the cities, and 

+ find out the maximum distance.

First of all, we need to go to the ``cities`` folder for this example.

In [ ]:
cd cities

We have prepared a Python module named `dist_cities` that help with reading city data and generating geographical coordinate pairs.

To read city data, type

In [ ]:
import dist_cities as dc
cities = dc.read_cities()
print("There are %d cities." % len(cities))
print("First city is:", cities[0])
print("Second city is:", cities[1])

The geographical coordinate pairs are generated by 

In [ ]:
# A coordinate pair is a tuple (latitude_1, latitude_2, longitude_1, longitude_2)
coord_pairs = dc.create_coord_pairs(cities)
print("There are %d coordinate pairs." % len(coord_pairs))
print("First coordinate pair is:", coord_pairs[0])

We provide a function for computing the distance of a geographical coordinate pair

In [ ]:
import math

def calc_dist(coord_pair):

    """Calculate the distance from a coordinate pair (latitude_1, latitude_2,
    longitude_1, longitude_2)"""

    p1 = coord_pair[0] / 180.0 * math.pi
    p2 = coord_pair[1] / 180.0 * math.pi

    cp1 = math.cos(p1)
    cp2 = math.cos(p2)

    dp = p2 - p1

    dL = (coord_pair[2] - coord_pair[3]) / 180.0 * math.pi

    a = math.sin(dp/2) **2 + cp1 * cp2 * math.sin(dL/2) **2
    c = 2.0 * math.atan2(math.sqrt(a), math.sqrt(1.0 - a))

    R = 6371 # radius of Earth in km

    return R*c

Let's double check the `DirectView` instance

In [ ]:
print(dview)

Since the `math` library is used, we need to import the library for each engine

In [ ]:
dview.execute('import math')

**Task:** Provide `dview.map_sync` the function and the list of input argument.

In [ ]:
# Write your parallel python via dview.map_sync
output = 

`output` contains all the distances. What is the maximum distance?

In [ ]:
print(max(output))

**Task:** Use the `%%timeit` magic to time your parallel calculation.

In [ ]:
# time your parallel calculation
%%timeit
output = 
print(max(output))

**Task:** Also time your serial calculation.

In [ ]:
# time your serial calculation
%%timeit
output = 
print(max(output))

## Multiprocessing in Python

Another way to run parallel calculation in Python is the `multiprocessing` module, 
which is a built-in module within the core Python modules and does not need any further installation.
We are going to briefly introduce the `Pool` submodule.

Make sure you are still in the `cities` folder.

In [ ]:
pwd

The processes for the `multiprocessing` module can be started by initializing a `Pool` instance with the number of processes

In [ ]:
import multiprocessing as mp
nprocs = 4
pool = mp.Pool(nprocs)

The `Pool` instance has a `map` function that works similarly to the `map_sync` function from `ipyparallel`.

**Task:** Provide the function and list of arguments to the `map` funciton of the `Pool` instance, and time it.

In [ ]:
%%timeit
output = 
print(max(output))

In [ ]:
for nprocs in range(1,24):
    pool = mp.Pool(nprocs)
    %timeit output = pool.map(calc_dist, coord_pairs)

Limitation of the `multiprocessing` module:

+ The spawned processes are bound to a single node and therefore not suitable for large-scale parapllelization on distributed memory system.

> The latest documentation for the multiprocessing module is here: https://docs.python.org/3.7/library/multiprocessing.html

## MPI4Py

### Introduction to MPI4Py

text

### MPI4Py basics

With MPI4Py, it is convenient to obtain the basic MPI settings including the communicator, the rank of the process, and the number of processes.

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

MPI4Py also provides convenient communicating functions like `send`, `recv`, `sacatter`, `gather`, etc.

There's no automatic mapping but you may find `scatter` and `gather` very useful in practice.

### Parallelization via MPI4Py

Below is an example code of calculating the distances via `mpi4py`.

In [ ]:
import time
from mpi4py import MPI

import dist_cities as dc

# MPI settings

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nprocs = comm.Get_size()

# prepare data and determine workloads

if rank == 0:
    cities = dc.read_cities()
    coord_pairs = dc.create_coord_pairs(cities)
    npairs = len(coord_pairs)

    dn = npairs // nprocs
    if npairs % nprocs != 0:
        dn += 1

# compute via MPI
# 1. Slice coord_pairs for processes
# 2. Scatter the sliced pieces
# 3. Do computation on each process
# 4. Gather results to master process
# 5. Collect the results into one list

t0 = time.time()

if rank == 0:
    data = [coord_pairs[int(x*dn):int((x+1)*dn)] for x in range(nprocs)]
else:
    data = None

data = comm.scatter(data, root=0)

result = [dc.calc_dist(p) for p in data]

result = comm.gather(result, root=0)

if rank == 0:
    output = []
    for a in result:
        output += a

t1 = time.time()

if rank == 0:
    print("Maximum distance: %.0f km" % max(output))
    print("Computing time: %.3f sec" % (t1-t0))

In [ ]:
%%px?

**NB: mpi4py can be used with ipython clusters, see: https://ipython.org/ipython-doc/3/parallel/parallel_mpi.html**

Note that, however, MPI4Py should be launched by `mpirun` and is usually executed outside of Jupyter notebook.

The `%salloc` magic allows the user to run MPI4Py job inside Jupyter notebook

**Task:** Copy-paste the code into a `.py` file and run the code by `%salloc` and `mpirun`.